In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("~/Desktop/data.csv")

In [3]:
data.head(5)

,section,date_mm_dd_yy,time,duration,name,break_activity,objection,synthesis,clarifying,building_comment,new,reference_to_text,interruption_side_talking,chaos_erupts
0,1,100318.0,10:22:21,NaN,joseph,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,100318.0,10:22:49,0:00:26,everyone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,100318.0,10:23:15,0:00:34,jehnna,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,100318.0,10:23:49,0:00:17,silence,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,100318.0,10:24:06,0:00:11,everyone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
sorted_data = data["name"].value_counts()
sorted_data = pd.DataFrame({"name": sorted_data.index, "count": sorted_data.values})
# print(sorted_data)

one_ofs = sorted_data["count"] <= 2
one_ofs = sorted_data[one_ofs]

In [5]:
def contribution_value(df, total):
    value = 0
    for i in df["count"]:
        value += i ** 2
    value /= total
    value /= len(df)
    return value

In [6]:
def filter_section(data, section):
    def class_list(row):
        drop = ["lee","jehnna","everyone","teachers","class","silence","various"]
        no_drop = ["casey","eli","molly","alice","kate","anam","anna","zulie","jojo"]
        if (row["name"] in drop) or (row["count"] <= 2 and row["name"] not in no_drop):
            return False
        else:
            return True
    section_index = data["section"] == section
    filtered = data[section_index]
    
    filtered_counts = filtered["name"].value_counts()
    filtered_counts = pd.DataFrame({"name": filtered_counts.index, "count": filtered_counts.values})
    
    filtered_counts["drop"] = filtered_counts.apply(class_list,axis=1)
    filtered_counts = filtered_counts[filtered_counts["drop"]]
    
    return [filtered_counts, len(filtered)]

In [7]:
sections = {}
for i in range(1,7):
    sections[i] = contribution_value(
                                    filter_section(data, i)[0],
                                    filter_section(data, i)[1],
                                    )
sections

{1: 1.2004435526454746,
 2: 1.712101021638429,
 3: 0.7011011301072153,
 4: 1.8614825240441004,
 5: 1.3858496332518337,
 6: 2.7601122129436324}

In [8]:
filter_section(data, 6)

[        name  count  drop
 1     amelia     97  True
 2       evan     96  True
 3     elliot     83  True
 4     amalia     80  True
 7   mirielle     58  True
 8    matthew     52  True
 9      jonah     37  True
 10   annie_z     33  True
 11  isabelle     30  True
 13     quetz     17  True
 14      ryan     16  True
 15      luke     14  True
 16       gus     11  True
 17     paige     10  True
 20     molly      2  True
 25     casey      1  True, 958]

In [9]:
section_index = data["section"] == 6
section_6 = data[section_index]
section_6

def constructive(row):
    if (row["interruption_side_talking"] == True or row["chaos_erupts"]):
        return False
    elif (row["name"] in ["lee", "jehnna", "leeandjehnna", "teachers"]):
        return False
    elif (row["name"] in ["everyone", "video", "activity", "workperiod"]):
        return False
    else:
        return True
section_6["constructive"] = section_6.apply(constructive, axis=1)
section_6 = section_6[["name","constructive"]]
section_6 = section_6.reset_index()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
for name in ["isabelle","amelia","ryan","elliot","molly","casey","amalia","luke","quetz","paige","jonah","matthew",
            "evan","gus","mirielle","annie_z"]:
    total_contributions = 0
    constructive_contributions = 0
    after_unconstructive = 0
    for i in range(len(section_6)):
        if (section_6.iloc[i]["name"] == name):
            if (section_6.iloc[i]["constructive"]):
                try:
                    if (not(section_6.iloc[i-1]["constructive"])):
                        after_unconstructive += 1
                except IndexError:
                    pass
                constructive_contributions += 1
            total_contributions += 1
    contribution_percent = round(after_unconstructive/total_contributions*100,2)
    constructive_percent = round(constructive_contributions/total_contributions*100,2)
    print("{} made {} contributions, {} of which were constructive, or {}%".format(
        name, total_contributions, constructive_contributions, constructive_percent))
    print("Out of their {} total constructive contributious, {}, or {}% followed an unconstructive action\n".format(
        constructive_contributions, after_unconstructive, contribution_percent))

isabelle made 30 contributions, 26 of which were constructive, or 86.67%
Out of their 26 total constructive contributious, 14, or 46.67% followed an unconstructive action

amelia made 97 contributions, 77 of which were constructive, or 79.38%
Out of their 77 total constructive contributious, 38, or 39.18% followed an unconstructive action

ryan made 16 contributions, 4 of which were constructive, or 25.0%
Out of their 4 total constructive contributious, 2, or 12.5% followed an unconstructive action

elliot made 83 contributions, 56 of which were constructive, or 67.47%
Out of their 56 total constructive contributious, 27, or 32.53% followed an unconstructive action

molly made 2 contributions, 2 of which were constructive, or 100.0%
Out of their 2 total constructive contributious, 1, or 50.0% followed an unconstructive action

casey made 1 contributions, 1 of which were constructive, or 100.0%
Out of their 1 total constructive contributious, 0, or 0.0% followed an unconstructive action